In [1]:
import gensim
import pandas as pd
import ast
from collections import defaultdict

In [2]:
data = pd.read_csv('subneighborhood_separated_articles/2018.csv')

In [3]:
data = data.drop(['Unnamed: 0'], axis=1)
type(data.iloc[0]['wellington_hill'])

str

In [4]:
black_sub_neighborhoods = ['wellington_hill', 'southern_mattapan', 'west_codman_hill', 'west_lower_mills', 'washington_hill',
                          'mattapan', 'franklin_field_south', 'dorchester_center', 'dorchester', 'bowdoin_north', 'mount_bowdoin',
                          'west_street', 'river_street', 'dudley', 'brunswick_king', 'nubian_square', 'roxbury', 'grove_hall',
                          'codman_square', 'upham\'s_corner', 'meeting_house_hill', 'washington_park', 'hyde_park', 'jones_hill',
                          'milton_hill', 'forest_hills', 'woodbourne', 'lower_roxbury', 'fields_corner_wester',
                          'frederick_douglas_square_historic_district', 'georgetown', 'st._marks', 'fort_hill', 'ashcroft',
                          'ashmount', 'clam_point', 'neponset', 'port_norfolk', 'columbus_park', 
                          'andrew_square', 'adams_village']

white_sub_neighborhoods = ['metropolitan_hill', 'beech_street', 'washington', 'mount_hope', 'd_street', 'west_broadway', 
                          'central_maverick_square', 'paris_square', 'bunker_hill', 'high_street_hill', 'roslindale',
                          'eagle_hill', 'columbus', 'central_village', 'east_boston',
                          'shawmut', 'harbor_view', 'orient_heights', 'mission_hill', 'south_end', 'jeffries_point', 
                          'upper_washington', 'spring_street', 'charlestown', 'stonybrook', 'bay_village', 'city_point',
                          'jamaica_plain', 'back_bay', 'brighton', 'egleston_square', 'west_fens', 'allston', 'lower_allston',
                          'hunnwell_hill', 'salisbury_road-corey_farm', 'boston', 'fenway-kenmore', 'oak_square', 
                          'medford_street', 'the_neck', 'brook_farm', 'seaport_district', 'centre_south',
                          'st._elizabeth\'s', 'china_town', 'aberdeen', 'bellevue_hill', 'downtown_crossing', 
                          'chestnut_hill_neighborhood_association', 'back_bay_west', 'west_end', 'telegraph_hill', 
                          'beachmont']

In [5]:
data = data.fillna("('no article', 'no_id')")
for col in data.columns:
    data[col] = data[col].apply(ast.literal_eval)

In [6]:
cols = data.columns
for col in cols:
    if data.iloc[0][col] == ('no article', 'no_id'):
        data = data.drop([col], axis=1)

In [7]:
rem_cols = data.columns

In [8]:
black_subs = 0
white_subs = 0
for col in rem_cols:
    if col in black_sub_neighborhoods:
        black_subs += 1
    else:
        white_subs += 1
print('Black Sub-neighborhoods = ' + str(black_subs))
print('White Sub-neighborhoods = ' + str(white_subs))

Black Sub-neighborhoods = 9
White Sub-neighborhoods = 5


In [9]:
documents = []
tot_cols = len(rem_cols)
for row in data.itertuples(index=False):
    for ind in range(tot_cols):
        if row[ind][0] != ('no article', 'no_id'):
            temp = row[ind][0].lower().split()
            
            length = len(temp) - 1
            num_mentions = {'wellington_hill': 0, 'southern_mattapan': 0, 'west_codman_hill': 0, 'west_lower_mills': 0, 'washington_hill': 0,
                        'mattapan': 0, 'franklin_field_south': 0, 'dorchester_center': 0, 'dorchester': 0, 'bowdoin_north': 0, 'mount_bowdoin': 0,
                        'west_street': 0, 'river_street': 0, 'dudley': 0, 'brunswick_king': 0, 'nubian_square': 0, 'roxbury': 0, 'grove_hall': 0,
                        'codman_square': 0, 'upham\'s_corner': 0, 'meeting_house_hill': 0, 'washington_park': 0, 'hyde_park': 0, 'jones_hill': 0,
                        'milton_hill': 0, 'forest_hills': 0, 'woodbourne': 0, 'lower_roxbury': 0, 'fields_corner_wester': 0,
                        'frederick_douglas_square_historic_district': 0, 'georgetown': 0, 'st._marks': 0, 'fort_hill': 0, 'ashcroft': 0,
                        'ashmount': 0, 'clam_point': 0, 'neponset': 0, 'port_norfolk': 0, 'columbus_park': 0, 
                        'andrew_square': 0, 'adams_village': 0, 'metropolitan_hill': 0, 'beech_street': 0, 'washington': 0, 'mount_hope': 0, 
                        'd_street': 0, 'west_broadway': 0, 'central_maverick_square': 0, 'paris_square': 0, 'bunker_hill': 0, 'high_street_hill': 0,
                        'roslindale': 0, 'eagle_hill': 0, 'columbus': 0, 'central_village': 0, 'east_boston': 0,
                        'shawmut': 0, 'harbor_view': 0, 'orient_heights': 0, 'mission_hill': 0, 'south_end': 0, 'jeffries_point': 0, 
                        'upper_washington': 0, 'spring_street': 0, 'charlestown': 0, 'stonybrook': 0, 'bay_village': 0, 'city_point': 0,
                        'jamaica_plain': 0, 'back_bay': 0, 'brighton': 0, 'egleston_square': 0, 'west_fens': 0, 'allston': 0, 'lower_allston': 0,
                        'hunnwell_hill': 0, 'salisbury_road-corey_farm': 0, 'boston': 0, 'fenway-kenmore': 0, 'oak_square': 0, 
                        'medford_street': 0, 'the_neck': 0, 'brook_farm': 0, 'seaport_district': 0, 'centre_south': 0,
                        'st._elizabeth\'s': 0, 'china_town': 0, 'aberdeen': 0, 'bellevue_hill': 0, 'downtown_crossing': 0, 
                        'chestnut_hill_neighborhood_association': 0, 'back_bay_west': 0, 'west_end': 0, 'telegraph_hill': 0, 'beachmont': 0}
            for i in range(length):
                if temp[i] == 'wellington' and temp[i + 1] == 'hill':
                    temp[i] = 'wellington_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['wellington_hill'] += 1
                elif temp[i] == 'southern' and temp[i + 1] == 'mattapan':
                    temp[i] = 'southern_mattapan'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['southern_mattapan'] += 1
                elif i > 0 and temp[i - 1] == 'west' and temp[i] == 'codman' and temp[i + 1] == 'hill':
                    temp[i - 1] = 'west_codman_hill'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['west_codman_hill'] += 1
                elif i > 0 and temp[i - 1] == 'west' and temp[i] == 'lower' and temp[i + 1] == 'mills':
                    temp[i - 1] = 'west_lower_mills'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['west_lower_mills'] += 1
                elif temp[i] == 'washington' and temp[i + 1] == 'hill':
                    temp[i] = 'washington_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['washington_hill'] += 1
                elif temp[i] == 'mattapan':
                    num_mentions['mattapan'] += 1
                elif i > 0 and temp[i - 1] == 'franklin' and temp[i] == 'field' and temp[i + 1] == 'south':
                    temp[i - 1] = 'franklin_field_south'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['franklin_field_south'] += 1
                elif temp[i] == 'dorchester' and temp[i + 1] == 'center':
                    temp[i] = 'dorchester_center'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['dorchester_center'] += 1
                elif temp[i] == 'dorchester':
                    num_mentions['dorchester'] += 1
                elif temp[i] == 'bowdoin' and temp[i + 1] == 'north':
                    temp[i] = 'bowdoin_north'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['bowdoin_north'] += 1
                elif temp[i] == 'mount' and temp[i + 1] == 'bowdoin':
                    temp[i] = 'mount_bowdoin'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['mount_bowdoin'] += 1
                elif temp[i] == 'west' and temp[i + 1] == 'street':
                    temp[i] = 'west_street'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['west_street'] += 1
                elif temp[i] == 'river' and temp[i + 1] == 'street':
                    temp[i] = 'river_street'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['river_street'] += 1
                elif temp[i] == 'dudley':
                    num_mentions['dudley'] += 1
                elif temp[i] == 'brunswick' and temp[i + 1] == 'king':
                    temp[i] = 'brunswick_king'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['brunswick_king'] += 1
                elif temp[i] == 'nubian' and temp[i + 1] == 'square':
                    temp[i] = 'nubian_square'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['nubian_square'] += 1
                elif temp[i] == 'roxbury':
                    num_mentions['roxbury'] += 1
                elif temp[i] == 'grove' and temp[i + 1] == 'hall':
                    temp[i] = 'grove_hall'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['grove_hall'] += 1
                elif temp[i] == 'codman' and temp[i + 1] == 'square':
                    temp[i] = 'codman_square'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['codman_square'] += 1
                elif temp[i] == 'upham\'s' and temp[i + 1] == 'corner':
                    temp[i] = 'upham\'s_corner'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['upham\'s_corner'] += 1
                elif i > 0 and temp[i - 1] == 'meeting' and temp[i] == 'house' and temp[i + 1] == 'hill':
                    temp[i - 1] = 'meeting_house_hill'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['meeting_house_hill'] += 1
                elif temp[i] == 'washington' and temp[i + 1] == 'park':
                    temp[i] = 'washington_park'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['washington_park'] += 1
                elif temp[i] == 'hyde' and temp[i + 1] == 'park':
                    temp[i] = 'hyde_park'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['hyde_park'] += 1
                elif temp[i] == 'jones' and temp[i + 1] == 'hill':
                    temp[i] = 'jones_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['jones_hill'] += 1
                elif temp[i] == 'milton' and temp[i + 1] == 'hill':
                    temp[i] = 'milton_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['milton_hill'] += 1
                elif temp[i] == 'forest' and temp[i + 1] == 'hills':
                    temp[i] = 'forest_hills'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['forest_hills'] += 1
                elif temp[i] == 'woodbourne':
                    num_mentions['woodbourne'] += 1
                elif temp[i] == 'lower' and temp[i + 1] == 'roxbury':
                    temp[i] = 'lower_roxbury'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['lower_roxbury'] += 1
                elif i > 0 and temp[i - 1] == 'fields' and temp[i] == 'corner' and temp[i + 1] == 'wester':
                    temp[i - 1] = 'fields_corner_wester'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['fields_corner_wester'] += 1
                elif i > 1 and temp[i - 2] == 'frederick' and temp[i - 1] == 'douglas' and temp[i] == 'square' and temp[i + 1] == 'historic' and temp[i + 2] == 'district':
                    temp[i - 2] = 'frederick_douglas_square_historic_district'
                    temp.pop(i + 2)
                    temp.pop(i + 1)
                    temp.pop(i)
                    temp.pop(i - 1)
                    length -= 4
                    num_mentions['frederick_douglas_square_historic_district'] += 1
                elif temp[i] == 'georgetown':
                    num_mentions['georgetown'] += 1
                elif temp[i] == 'st.' and temp[i + 1] == 'marks':
                    temp[i] = 'st._marks'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['st._marks'] += 1
                elif temp[i] == 'fort' and temp[i + 1] == 'hill':
                    temp[i] = 'fort_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['fort_hill'] += 1
                elif temp[i] == 'ashcroft':
                    num_mentions['ashcroft'] += 1
                elif temp[i] == 'ashmount':
                    num_mentions['ashmount'] += 1
                elif temp[i] == 'clam' and temp[i + 1] == 'point':
                    temp[i] = 'clam_point'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['clam_point'] += 1
                elif temp[i] == 'port' and temp[i + 1] == 'norfolk':
                    temp[i] = 'port_norfolk'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['port_norfolk'] += 1
                elif temp[i] == 'neponset':
                    num_mentions['neponset'] += 1
                elif temp[i] == 'columbus' and temp[i + 1] == 'park':
                    temp[i] = 'columbus_park'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['columbus_park'] += 1
                elif temp[i] == 'andrew' and temp[i + 1] == 'square':
                    temp[i] = 'andrew_square'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['andrew_square'] += 1
                elif temp[i] == 'adams' and temp[i + 1] == 'village':
                    temp[i] = 'adams_village'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['adams_village'] += 1
                elif temp[i] == 'metropolitan' and temp[i + 1] == 'hill':
                    temp[i] = 'metropolitan_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['metropolitan_hill'] += 1
                elif temp[i] == 'beech' and temp[i + 1] == 'street':
                    temp[i] = 'beech_street'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['beech_street'] += 1
                elif temp[i] == 'washington':
                    num_mentions['washington'] += 1
                elif temp[i] == 'mount' and temp[i + 1] == 'hope':
                    temp[i] = 'mount_hope'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['mount_hope'] += 1
                elif temp[i] == 'd' and temp[i + 1] == 'street':
                    temp[i] = 'd_street'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['d_street'] += 1
                elif temp[i] == 'west' and temp[i + 1] == 'broadway':
                    temp[i] = 'west_broadway'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['west_broadway'] += 1
                elif i > 0 and temp[i - 1] == 'central' and temp[i] == 'maverick' and temp[i + 1] == 'square':
                    temp[i - 1] = 'central_maverick_square'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['central_maverick_square'] += 1
                elif temp[i] == 'paris' and temp[i + 1] == 'square':
                    temp[i] = 'paris_square'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['paris_square'] += 1
                elif temp[i] == 'bunker' and temp[i + 1] == 'hill':
                    temp[i] = 'bunker_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['bunker_hill'] += 1
                elif i > 0 and temp[i - 1] == 'high' and temp[i] == 'street' and temp[i + 1] == 'hill':
                    temp[i - 1] = 'high_street_hill'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['high_street_hill'] += 1
                elif temp[i] == 'roslindale':
                    num_mentions['roslindale'] += 1
                elif temp[i] == 'eagle' and temp[i + 1] == 'hill':
                    temp[i] = 'eagle_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['eagle_hill'] += 1
                elif temp[i] == 'columbus':
                    num_mentions['columbus'] += 1
                elif temp[i] == 'central' and temp[i + 1] == 'village':
                    temp[i] = 'central_village'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['central_village'] += 1
                elif temp[i] == 'east' and temp[i + 1] == 'boston':
                    temp[i] = 'east_boston'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['east_boston'] += 1
                elif temp[i] == 'shawmut':
                    num_mentions['shawmut'] += 1
                elif temp[i] == 'harbor' and temp[i + 1] == 'view':
                    temp[i] = 'harbor_view'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['harbor_view'] += 1
                elif temp[i] == 'orient' and temp[i + 1] == 'heights':
                    temp[i] = 'orient_heights'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['orient_heights'] += 1
                elif temp[i] == 'mission' and temp[i + 1] == 'hill':
                    temp[i] = 'mission_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['mission_hill'] += 1
                elif temp[i] == 'south' and temp[i + 1] == 'end':
                    temp[i] = 'south_end'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['south_end'] += 1
                elif temp[i] == 'jeffries' and temp[i + 1] == 'point':
                    temp[i] = 'jeffries_point'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['jeffries_point'] += 1
                elif temp[i] == 'upper' and temp[i + 1] == 'washington':
                    temp[i] = 'upper_washington'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['upper_washington'] += 1
                elif temp[i] == 'spring' and temp[i + 1] == 'street':
                    temp[i] = 'spring_street'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['spring_street'] += 1
                elif temp[i] == 'charlestown':
                    num_mentions['charlestown'] += 1
                elif temp[i] == 'stonybrook':
                    num_mentions['stonybrook'] += 1
                elif temp[i] == 'bay' and temp[i + 1] == 'village':
                    temp[i] = 'bay_village'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['bay_village'] += 1
                elif temp[i] == 'city' and temp[i + 1] == 'point':
                    temp[i] = 'city_point'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['city_point'] += 1
                elif temp[i] == 'jamaica' and temp[i + 1] == 'plain':
                    temp[i] = 'jamaica_plain'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['jamaica_plain'] += 1
                elif temp[i] == 'back' and temp[i + 1] == 'bay':
                    temp[i] = 'back_bay'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['back_bay'] += 1
                elif temp[i] == 'brighton':
                    num_mentions['brighton'] += 1
                elif temp[i] == 'egleston' and temp[i + 1] == 'square':
                    temp[i] = 'egleston_square'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['egleston_square'] += 1
                elif temp[i] == 'west' and temp[i + 1] == 'fens':
                    temp[i] = 'west_fens'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['west_fens'] += 1
                elif temp[i] == 'allston':
                    num_mentions['allston'] += 1
                elif temp[i] == 'lower' and temp[i + 1] == 'allston':
                    temp[i] = 'lower_allston'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['lower_allston'] += 1
                elif temp[i] == 'hunnwell' and temp[i + 1] == 'hill':
                    temp[i] = 'hunnwell_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['hunnwell_hill'] += 1
                elif i > 0 and temp[i - 1] == 'salisbury' and temp[i] == 'road-corey' and temp[i + 1] == 'farm':
                    temp[i - 1] = 'salisbury_road-corey_farm'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['salisbury_road-corey_farm'] += 1
                elif temp[i] == 'boston':
                    num_mentions['boston'] += 1
                elif temp[i] == 'fenway-kenmore':
                    num_mentions['fenway-kenmore'] += 1
                elif temp[i] == 'oak' and temp[i + 1] == 'square':
                    temp[i] = 'oak_square'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['oak_square'] += 1
                elif temp[i] == 'medford' and temp[i + 1] == 'street':
                    temp[i] = 'medford_street'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['medford_street'] += 1
                elif temp[i] == 'the' and temp[i + 1] == 'neck':
                    temp[i] = 'the_neck'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['the_neck'] += 1
                elif temp[i] == 'brook' and temp[i + 1] == 'farm':
                    temp[i] = 'brook_farm'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['brook_farm'] += 1
                elif temp[i] == 'seaport' and temp[i + 1] == 'district':
                    temp[i] = 'seaport_district'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['seaport_district'] += 1
                elif temp[i] == 'centre' and temp[i + 1] == 'south':
                    temp[i] = 'centre_south'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['centre_south'] += 1
                elif temp[i] == 'st.' and temp[i + 1] == 'elizabeth\'s':
                    temp[i] = 'st._elizabeth\'s'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['st._elizabeth\'s'] += 1
                elif temp[i] == 'china' and temp[i + 1] == 'town':
                    temp[i] = 'china_town'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['china_town'] += 1
                elif temp[i] == 'aberdeen':
                    num_mentions['aberdeen'] += 1
                elif temp[i] == 'bellevue' and temp[i + 1] == 'hill':
                    temp[i] = 'bellevue_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['bellevue_hill'] += 1
                elif temp[i] == 'downtown' and temp[i + 1] == 'crossing':
                    temp[i] = 'downtown_crossing'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['downtown_crossing'] += 1
                elif i > 1 and temp[i - 2] == 'chestnut' and temp[i - 1] == 'hill' and temp[i] == 'neighborhood' and temp[i + 1] == 'association':
                    temp[i - 2] = 'chestnut_hill_neighborhood_association'
                    temp.pop(i + 1)
                    temp.pop(i)
                    temp.pop(i - 1)
                    length -= 3
                    num_mentions['chestnut_hill_neighborhood_association'] += 1
                elif i > 0 and temp[i - 1] == 'back' and temp[i] == 'bay' and temp[i + 1] == 'west':
                    temp[i - 1] = 'back_bay_west'
                    temp.pop(i + 1)
                    temp.pop(i)
                    length -= 2
                    num_mentions['back_bay_west'] += 1
                elif temp[i] == 'west' and temp[i + 1] == 'end':
                    temp[i] = 'west_end'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['west_end'] += 1
                elif temp[i] == 'telegraph' and temp[i + 1] == 'hill':
                    temp[i] = 'telegraph_hill'
                    temp.pop(i + 1)
                    length -= 1
                    num_mentions['telegraph_hill'] += 1
                elif temp[i] == 'beachmont':
                    num_mentions['beachmont'] += 1
                elif i >= length - 3:
                    break
            
            documents.append(temp)

In [10]:
# create Word2Vec model
# the skip-grams method is used here, with a window of 10
model = gensim.models.Word2Vec(window=10, min_count=2, sg=1, workers=10)
model.build_vocab(documents)  # prepare the model vocabulary

# train model on available data
# I use 5 epochs since that's standard
model.train(corpus_iterable=documents, total_examples=len(documents), epochs=5)

(4237533, 5470960)

In [11]:
from collections import defaultdict

In [12]:
keywords = rem_cols

# finding similar words and creating a csv file
# check if similar words list includes any keywords
# if so, remove those keywords from the similar words list

def compute_similar_words(model,source_word, topn=25):
    similar_words = [source_word]
    try:
        top_words = model.wv.most_similar(source_word, topn=topn)
        length_top = len(top_words)
        i = 0
        while i < length_top:
            if i >= length_top - 1:
                break
            elif top_words[i][0] in keywords:
                top_words.pop(i)
                length_top -= 1
            else:
                i += 1
        similar_words.extend([val[0] for val in top_words[:5]])
    except KeyError as err:
        print(err.args)
    return similar_words    

def compute_similar_words_and_representative_for_all_tasks(model, topn=25):
    columns = ['word' + str(i - 1) for i in range(1, 7)]
    df = pd.DataFrame(data=None, columns=columns)
    reps = defaultdict()
    for source_word in keywords:
        similar_words = compute_similar_words(model, source_word, topn)
        
        df.loc[len(df)] = similar_words
        sum_vec = (model.wv[similar_words[0]]).copy()
        for i in range(1, 5):
            temp = model.wv[similar_words[i]]
            sum_vec += temp.copy()
        rep_vec = sum_vec / 5
        reps[source_word] = model.wv.most_similar(positive=[rep_vec], topn=1)
    df.to_csv('similar_words_task/sub_neighborhood_similar_words/2018_findings.csv')
    rep_ser = pd.Series(reps)
    rep_ser.to_csv('similar_words_task/sub_neighborhood_representative_words/2018_sub_neighborhoods.csv')
    
words = compute_similar_words_and_representative_for_all_tasks(model)